<div >
<img src = "../banner.jpg" />
</div>

<a target="_blank" href="https://colab.research.google.com/github/ignaciomsarmiento//BDML_202302/blob/main/Lecture08/Notebook_SS08_Class.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Generative Models for Classification 

To work through the steps of probability-based classification, we’ll use a real dataset on loans and credit from a set of local lenders in Germany (taken from the UC Irvine Machine Learning Repository and cleaned for our purposes). 

Credit scoring is a classic problem of classification, and it remains one of the big application domains for ML: use previous loan results (default versus payment) to train a model that can predict the performance of potential new loans.

\begin{align}
Default=f(x) + u
\end{align}

where $Default=I(Default=1)$

Taken from the UC Irvine Machine Learning Repository.


In [1]:
#Cargar librerías 
require("pacman")
p_load(tidyverse)
set.seed(1011)

Loading required package: pacman



In [2]:
#Leer los datos 
credit <- readRDS(url("https://github.com/ignaciomsarmiento/datasets/blob/main/credit_class.rds?raw=true"))
head(credit)   


,Default,duration,amount,installment,age,history,purpose,foreign,rent
,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>
1,0,6,1169,4,67,terrible,goods/repair,foreign,FALSE
2,1,48,5951,2,22,poor,goods/repair,foreign,FALSE
3,0,12,2096,2,49,terrible,edu,foreign,FALSE
4,0,42,7882,2,45,poor,goods/repair,foreign,FALSE
5,1,24,4870,3,53,poor,newcar,foreign,FALSE
6,0,36,9055,2,35,poor,edu,foreign,FALSE


In [3]:
default<-credit$Default  #defino ahora va a servir después

#mutación de factores
credit<-credit %>% mutate(Default=factor(Default,levels=c(0,1),labels=c("No","Si")),
                          history=factor(history,levels=c("good","poor","terrible"),labels=c("buena","mala","terrible")),
                          foreign=factor(foreign,levels=c("foreign","german"),labels=c("extranjero","aleman")),
                          purpose=factor(purpose,levels=c("newcar","usedcar","goods/repair","edu", "biz" ),labels=c("auto_nuevo","auto_usado","bienes","educacion","negocios")))         

## Out of sample prediction

In [4]:
p_load("caret")

inTrain <- createDataPartition(
  y = credit$Default,## La variable dependiente u objetivo 
  p = .7, ## Usamos 70%  de los datos en el conjunto de entrenamiento 
  list = FALSE)


train <- credit[ inTrain,]
test  <- credit[-inTrain,]

In [ ]:
head(train)

## LDA


\begin{align}
p (Y=1|X)=\frac{f(X|Y=1)\pi(Y=1)}{m(X)}
\end{align}


with $m(X)$ is the marginal distribution of $X$, since there are two states of nature $y \rightarrow i\in\{0,1\}$

\begin{align}
m(X) &= f(X|Y=1)\pi(Y=1) + f(X|Y=0)\pi(Y=0) 
\end{align}


\begin{align}
m(X)     &= f(X|Y=1)\pi(Y=1) + f(X|Y=0)(1-\pi(Y=1))
\end{align}

We need to estimate 
   - $\pi(Y=1)$ 
   - $f(X|Y=1)$,  
   - $f(X|Y=0)$ 



#### By Hand (k=1)

Let's implement LDA using duration as a predictor


- Let's start by estimating $\pi(Y=1)$. We've done this before

    \begin{align}
    \pi(Y=1) = \frac{\sum_{i=1}^n 1[Y_i=1]}{N}
    \end{align}


In [5]:
pi1<-sum(train$Default=="Si")/dim(train)[1]
pi1

[1] 0.3

In [6]:
dim(train)[1]

[1] 700


- Next $f(X|Y=j)$ with $j=0,1$. 

    - If we assume one predictor and $X|Y\sim N(\mu_j,\sigma_j)$, the problem boils down to estimating $\mu_j,\sigma_j$

    - LDA makes it simpler, assumes $\sigma_j=\sigma$ $\forall j$

To do this partition the sample in two $Y=0$ and $Y=1$, estimate the moments and get $\hat{f}(X|Y=j)$

**Means**

\begin{align}
\hat{\mu}_k=\frac{1}{n_k}\sum_{i:y_i=k}x_i
\end{align}

In [7]:
#Means
mu1<-mean(train$duration[train$Default=="Si"])
mu1

[1] 24.17143

In [8]:
mu0<-mean(train$duration[train$Default=="No"])
mu0

[1] 19.2551

**Variance**

\begin{align}
\hat{\sigma}^2 = \frac{1}{N-K} \sum_{k=1}^K \sum_{i:y_i=k} (x_i -\hat{\mu}_k)^2
\end{align}

In [9]:
#Variance
g1<-sum((train$duration[train$Default=="Si"]-mu1)^2)
g0<-sum((train$duration[train$Default=="No"]-mu0)^2)


sigma<-sqrt((g1+g0)/(dim(train)[1]-2))
sigma

[1] 11.49118

With the moments, now we can obtain $f(X|Y=j)$ with $j=0,1$. 

In [10]:
f1<-dnorm(test$duration,mean=mu1,sd=sigma)
f0<-dnorm(test$duration,mean=mu0,sd=sigma)

- Finally plug everything into the Bayes Rule and we are done:
\begin{align}
p (Y=1|X)=\frac{f(X|Y=1)\pi(Y=1)}{f(X|Y=1)\pi(Y=1) + f(X|Y=0)(1-\pi(Y=1))}
\end{align}


In [11]:
post_hand<-f1*pi1/(f1*pi1+f0*(1-pi1))
head(post_hand)

[1] 0.1927363 0.5328026 0.3181777 0.4218021 0.4218021 0.3684739

In [12]:
p_load("MASS")     # LDA
lda_simple <- lda(Default~duration, data = train)
lda_simple_pred<-predict(lda_simple,test)
names(lda_simple_pred)


[1] "class"     "posterior" "x"

In [13]:
posteriors<-data.frame(lda_simple_pred$posterior)
posteriors$hand<-post_hand

head(posteriors)

,No,Si,hand
,<dbl>,<dbl>,<dbl>
1,0.8072637,0.1927363,0.1927363
2,0.4671974,0.5328026,0.5328026
5,0.6818223,0.3181777,0.3181777
6,0.5781979,0.4218021,0.4218021
8,0.5781979,0.4218021,0.4218021
10,0.6315261,0.3684739,0.3684739


### Caret

In [14]:
ctrl<- trainControl(method = "cv",
                    number = 5,
                    classProbs = TRUE,
                    verbose=FALSE,
                    savePredictions = T)

lda_fit = train(Default~duration+amount+installment+age, 
                data=train, 
                method="lda",
                trControl = ctrl)

lda_fit

Linear Discriminant Analysis 

700 samples
  4 predictor
  2 classes: 'No', 'Si' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 560, 560, 560, 560, 560 
Resampling results:

  Accuracy  Kappa    
  0.71      0.1134205


In [15]:
qda_fit = train(Default~duration+amount+installment+age, 
                data=train, 
                method="qda",
                trControl = ctrl)

qda_fit

Quadratic Discriminant Analysis 

700 samples
  4 predictor
  2 classes: 'No', 'Si' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 560, 560, 560, 560, 560 
Resampling results:

  Accuracy   Kappa    
  0.6985714  0.1746653


In [ ]:
head(credit)

## Naive Bayes

In [16]:
p_load("klaR")

set.seed(1410)
mylogit_nb <- train(Default~duration+amount+installment+age+history+purpose+foreign+rent,
                       data = train, 
                       method = "nb",
                       trControl = ctrl,
                        tuneGrid=expand.grid(fL=seq(0,10,length.out = 3),
                                            usekernel=TRUE,
                                            adjust=seq(1,10,length.out = 3)))



mylogit_nb

Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 1”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 2”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 3”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 4”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 5”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 6”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 7”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 8”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 9”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 10”
Warning message in 

Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 84”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 85”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 86”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 87”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 88”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 89”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 90”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 91”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 92”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 93”
Warning me

Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 27”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 28”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 29”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 30”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 31”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 32”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 33”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 34”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 35”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 36”
Warning me

Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 110”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 111”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 112”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 113”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 114”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 115”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 116”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 117”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 118”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 119”


Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 53”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 54”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 55”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 56”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 57”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 58”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 59”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 60”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 61”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 62”
Warning me

...

Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 131”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 132”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 133”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 134”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 135”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 136”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 137”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 138”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 139”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 140”


Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 74”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 75”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 76”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 77”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 78”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 79”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 80”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 81”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 82”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 83”
Warning me

Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 17”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 18”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 19”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 20”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 21”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 22”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 23”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 24”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 25”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 26”
Warning me

Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 100”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 101”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 102”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 103”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 104”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 105”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 106”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 107”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 108”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 109”


Naive Bayes 

700 samples
  8 predictor
  2 classes: 'No', 'Si' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 560, 560, 560, 560, 560 
Resampling results across tuning parameters:

  fL  adjust  Accuracy   Kappa     
   0   1.0    0.7057143  0.05102803
   0   5.5    0.7085714  0.06376805
   0  10.0    0.7042857  0.01981132
   5   1.0    0.7057143  0.05102803
   5   5.5    0.7085714  0.06376805
   5  10.0    0.7042857  0.01981132
  10   1.0    0.7057143  0.05102803
  10   5.5    0.7085714  0.06376805
  10  10.0    0.7042857  0.01981132

Tuning parameter 'usekernel' was held constant at a value of TRUE
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were fL = 0, usekernel = TRUE and adjust
 = 5.5.

## KNN

In [ ]:
set.seed(1410)
mylogit_knn <- train(Default~duration+amount+installment+age+
                       duration+amount+installment+age+history+purpose+foreign+rent,
                       data = train, 
                       method = "knn",
                       trControl = ctrl,
                     tuneGrid = expand.grid(k=c(3,5,7,9,11)))


mylogit_knn